In [1]:
from scipy.signal import find_peaks
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
import scipy.signal
from astropy import units as u
import nana
from scipy.interpolate import CubicSpline
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
from math import cos, sin, radians
from matplotlib.collections import EllipseCollection

In [2]:
def get_started(num_of_peaks, xs, ys): 
    """
    
    Identifies and returns the indices of the highest peaks in a given dataset.
    
    Args:
        num_of_peaks (int): The number of highest peaks to return.
        xs (numpy.ndarray): The x-axis values 
        ys (numpy.ndarray): The y-axis values

    Returns:
        numpy array: An array of indices corresponding to the highest peaks in `ys`.

    Bugs:
        `num_of_peaks` cannot be greater than the number of detected peaks
        `xs` or `ys` must be NumPy array

    
    """
    indxs, properties = find_peaks(ys)
    return indxs[np.argsort(-ys[indxs])[:num_of_peaks]]

def check_inputs(xs):
    """
    
    Checks whether the input array `xs` is sorted in ascending order.

    Args:
        xs (numpy.ndarray or list): The input array to check.

    Returns:
        bool: `True` if `xs` is sorted in ascending order, otherwise `False`.
    
    """
    for i in range(len(xs)-1):
        if xs[i] > xs[i+1]:
            print("check_inputs(): input xs is badly ordered. Use reorder_inputs to reorder")
            return False
    return True

def reorder_inputs(xs,ys):
    """
    
    Reorders the input arrays `xs` and `ys` in ascending order of `xs`.

    Args:
        xs (numpy.ndarray): The x-axis values 
        ys (numpy.ndarray): The y-axis values
        
    Returns:
        tuple of numpy arrays (sorted xs, sorted ys)

    Bugs:
        `xs` or `ys` must be NumPy array
        `xs` and `ys`must be same length
        
    """
    i = np.argsort(xs)
    return xs[i], ys[i]

#xlist is the teh index left to highest peak, highest peak, and the index right to teh highest peak
def design_matrix(xlist): 
    """
    
    Constructs a design matrix for quadratic curve fitting.

    Args:
        xs (numpy.ndarray): The x-axis values 

    Returns:
        3 x 3 design matrix of numpy arrays

    Bugs:
        xlist must be an numpy array

    Note:
        Note the factor of 0.5 that Hogg likes and no one else
        Also assumes xlist is ordered
    
    """
    return (np.vstack((xlist**0,xlist**1,0.5*xlist**2))).T

def fit_parabola(xs, ys, index):
    """

    Fits a quadratic function to three consecutive data points. Solves for coefficients (b,m,q) in the quadratic
    f(x) = b + m*x + (1/2) * q * x^2

    Args:
        xs (numpy.ndarray): The x-axis values 
        ys (numpy.ndarray): The y-axis values
        index (int): The index of peak
        

    Returns:
        tuple: (b, m, q)

    Bugs: 
        index-1` or `index+2` can't be out of bounds
        ``xs` or `ys` must be NumPy array
        
    
    """
    return np.linalg.solve(design_matrix(xs[index-1:index+2]), ys[index-1:index+2])

def refine_peak(xs, ys, index):
    """

    Refines the peak position using quadratic fit

    Args:
        xs (numpy.ndarray): The x-axis values 
        ys (numpy.ndarray): The y-axis values
        index (int): The index of peak
    
    Returns:
        3-tuple: (x position of refined peak, y position of refined peak, and second derivative (q))

    Bugs:
        Must be synchronized with the design matrix
        
        
    """
    b,m,q = fit_parabola(xs, ys, index)
    x_peak = -m / q
    return x_peak, 0.5 * q * (x_peak) ** 2 + m * (x_peak) + b, q
    
def refine_peaks(xs, ys, indices):
    """

    Refines the peak position for a set of indices using quadratic fit

    Args:
        xs (numpy.ndarray): The x-axis values 
        ys (numpy.ndarray): The y-axis values
        indices (numpy array): indices of the peaks (this should be the output of get_started()

    Returns:
        three  numpy arrays (array of refined x positions, array of refined y positions, and the second derivatives)
    

    """
    foo = lambda i: refine_peak(xs,ys,i)
    xs_refined, ys_refined, second_derivatives = zip(*list(map(foo,indices)))
    return np.array(xs_refined), np.array(ys_refined), np.array(second_derivatives)


def folding_freq(fs, ps, sampling_time, makeplots=False):
    """
    ##bugs:
    - assumes fs are ordered
    - global delta_f
    """
    fc_guess = 1. / sampling_time
    
    IA = fs < 0.5 * fc_guess
    fsA, psA = fs[IA], ps[IA]
    fsA,psA  = fsA[2:-2],  psA[2:-2]
    cs = CubicSpline(fs, ps, extrapolate=False)
    
    small, tiny = 20 * delta_f, 0.25 * delta_f
    fc_candidates = np.arange(fc_guess - small, fc_guess + small, tiny)
    foos_c = np.array([np.nansum(psA * cs(fc - fsA)) for fc in fc_candidates])
    fc_index = get_started(1,fc_candidates, foos_c)
    fc, _, _ = refine_peaks(fc_candidates, foos_c, fc_index)
    fc = fc[0]
    
    if makeplots:
        plt.plot(fc_candidates, foos_c)
        plt.axvline(fc_guess)
        plt.axvline(fc, color = 'red', alpha = 0.5)
        plt.title(fc)
        plt.show()

    
    return fc


In [3]:
search_result = lk.search_lightcurve('KIC 5202905', mission='Kepler')
lc_collection = search_result.download_all()

In [4]:
lc = lc_collection.stitch()
total_observation_time = (lc.time[-1] - lc.time[0]).value
delta_f = (1/total_observation_time) 
sampling_time= np.median(np.diff(lc.time.value))

In [5]:
f_max = (2 / (sampling_time))
f_min = delta_f/3
frequency_grid = np.arange(f_min, f_max, f_min)/(u.day)

In [6]:
pg = lc.to_periodogram(
    method='lombscargle',
    normalization='psd',
    frequency=frequency_grid
)

power = pg.power.value
freq = pg.frequency.to(1/u.day).value 

In [7]:
indices = get_started(10, freq, power)
refined_freq, _ , _ = refine_peaks(freq, power, indices)
nu = refined_freq[0]

In [8]:
t = lc.time.value
flux = lc.flux.value
sigma = lc.flux_err.value

In [9]:
t_clean = np.ma.filled(t, np.nan)
flux_clean = np.ma.filled(flux, np.nan)
sigma_clean = np.ma.filled(sigma, np.nan)

mask = np.isfinite(t_clean) & np.isfinite(flux_clean) & np.isfinite(sigma_clean)

lc = lc[mask]
t_fit = t_clean[mask]
flux_fit = flux_clean[mask]
sigma_fit = sigma_clean[mask]

In [10]:
lc_exptime = (6.52 * 270) / (60 * 60 * 24) #days, see Kepler Data Processing Handbook, Section 3.1
weight_fit = 1 / sigma_fit**2

def integral_design_matrix(ts, om, T):
    """
    ##bugs:
    - assumes all data points have the same exposure time, `T`
    - not numerically stable when `om * T` is small
    """
    return np.vstack([
        np.ones_like(ts),
        (+ np.sin(om * (ts + T/2)) - np.sin(om * (ts - T/2))) / (om * T),
        (- np.cos(om * (ts + T/2)) + np.cos(om * (ts - T/2))) / (om * T)
    ]).T

def weighted_least_squares_new(A, b, weights):
    ATA = A.T @ (A * weights[:, np.newaxis])
    ATb = A.T @ (b * weights)
    trace = np.trace(ATA)
    det = np.linalg.det(ATA)
    return np.linalg.solve(ATA, ATb), ATA, trace, det
    
def weighted_least_squares(A, b, weights):
    ATA = A.T @ (A * weights[:, np.newaxis])
    ATb = A.T @ (b * weights)
    return A @ np.linalg.solve(ATA, ATb)
    
def integral_chi_squared(om, ts, ys, ws, T):
    A = integral_design_matrix(ts, om, T)
    return np.sum(ws * (ys - weighted_least_squares(A, ys, ws))**2)

In [11]:
def check_coherence(ts, ys, weights, om):
    """
    assumes a lot of things about the data
    """
    cases = [(np.ones_like(ts).astype(bool), "all"),
             (ts < np.median(ts), "early"),
             (ts > np.median(ts), "late")]
    for I, name in cases:
        A = integral_design_matrix(ts[I], om, lc_exptime)
        pars, invar = weighted_least_squares(A, ys[I], weights[I])
        print(name, pars, invar)

In [12]:
def check_coherence(ts, ys, weights, om):
    """
    assumes a lot of things about the data
    """
    axes, As,Bs, traces, dets = [],[], [], [], []
    cases = [(np.ones_like(ts).astype(bool), "all"),
             (ts < np.median(ts), "early"),
             (ts > np.median(ts), "late")]
    for I, name in cases:
        A = integral_design_matrix(ts[I], om, lc_exptime)
        pars, invar, trace, det = weighted_least_squares_new(A, ys[I], weights[I])
        a,b = pars[1], pars[2]
        As.append(a)
        Bs.append(b)
        traces.append(trace)
        dets.append(det)
        covariance = np.linalg.inv(invar)
        covariance2 = covariance[1:, 1:]
        eigvals, eigvecs = np.linalg.eigh(covariance2)
        axis1 = np.sqrt(eigvals)*eigvecs[:, 0] 
        axis2 = np.sqrt(eigvals)*eigvecs[:, 1] 
        axes.append((axis1, axis2))
    return axes[0], axes[1], axes[2], As, Bs, traces, dets

In [13]:
all, early, late, As, Bs = check_coherence(t_fit, flux_fit, weight_fit, 2. * np.pi * nu)

ValueError: too many values to unpack (expected 5)

In [ ]:
_, _, _, _, _, traces, dets = check_coherence(t_fit, flux_fit, weight_fit, 2. * np.pi * nu)
print("trace all:", traces[0], "trace early:", traces[1], "trace late:", traces[2])
print("det all:", traces[0], "det early:", traces[1], "det late:", traces[2])

In [ ]:
#ELLIPSE-MAKING!!!!
es = []

#all
width_all, height_all = 2*np.linalg.norm(all[0]), 2*np.linalg.norm(all[1])
vec1 = all[0]
angle_all = np.degrees(np.arctan2(vec1[0], vec1[1]))
center = (As[0],Bs[0])
r = np.radians(angle_all)
ellipse_all = Ellipse(center, width_all, height_all, angle = angle_all, facecolor = 'none', edgecolor = 'black', label = "all")
es.append(ellipse_all)

#early
width_early, height_early = 2*np.linalg.norm(early[0]), 2*np.linalg.norm(early[1])
vec2 = early[0]
angle_early = np.degrees(np.arctan2(vec2[0], vec2[1]))
center = (As[1], Bs[1])
r = np.radians(angle_early)
ellipse_early = Ellipse(center, width_early, height_early, angle = angle_early, facecolor = 'none', edgecolor = 'blue', label = "early")
es.append(ellipse_early)

#late
width_late, height_late = 2*np.linalg.norm(late[0]), 2*np.linalg.norm(late[1])
vec3 = late[0]
angle_late = np.degrees(np.arctan2(vec3[0], vec3[1]))
center = (As[2], Bs[2])
r = np.radians(angle_late)
ellipse_late = Ellipse(center, width_late, height_late, angle = angle_late, facecolor = 'none', edgecolor = 'green', label = "late")
es.append(ellipse_late)

fig, ax = plt.subplots()
x_coords = []
y_coords = []

for e in es:
    cx, cy = e.center
    w, h = e.width / 2, e.height / 2
    x_coords.extend([cx - w, cx + w])
    y_coords.extend([cy - h, cy + h])
    
    # Compute limits with padding
x_pad = (max(x_coords) - min(x_coords)) * 0.05
y_pad = (max(y_coords) - min(y_coords)) * 0.05
    
xlim = (min(x_coords) - x_pad, max(x_coords) + x_pad)
ylim = (min(y_coords) - y_pad, max(y_coords) + y_pad)


for e in es:
    ax.add_artist(e)
    
ax.set_aspect('equal')
ax.grid(True)
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.legend(loc='best')
    
plt.show()

In [ ]:
print(center[0] - width_all, center[0] + width_all)
print(center[1] - height_all, center[1] + height_all)

In [ ]:
#more simplified way

es = []
cases = [("all", all, As[0], Bs[0]),
         ("early", early, As[1], Bs[1]),
         ("late", late, As[2], Bs[2])]

for label, (axis1, axis2), a, b in cases:
    width = 2 * np.linalg.norm(axis2)
    height = 2 * np.linalg.norm(axis1)
    angle = np.degrees(np.arctan2(axis2[1], axis2[0]))  
    center = (a, b)
    
    e = Ellipse(center, width, height, angle=angle,facecolor='none', edgecolor='green')
    es.append(e)

fig, ax = plt.subplots()
x_coords = []
y_coords = []

for e in es:
    cx, cy = e.center
    w, h = e.width / 2, e.height / 2
    x_coords.extend([cx - w, cx + w])
    y_coords.extend([cy - h, cy + h])
    
    # Compute limits with padding
x_pad = (max(x_coords) - min(x_coords)) * 0.05
y_pad = (max(y_coords) - min(y_coords)) * 0.05
    
xlim = (min(x_coords) - x_pad, max(x_coords) + x_pad)
ylim = (min(y_coords) - y_pad, max(y_coords) + y_pad)


for e in es:
    ax.add_artist(e)
    
ax.set_aspect('equal')
ax.grid(True)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

    
plt.show()

In [ ]:
#Adding vectors to the ellipses "all" case
# all, early, late, As, Bs = check_coherence(t_fit, flux_fit, weight_fit, 2. * np.pi * nu), this code was run in a cell before
#all, early, late are 2x2 matrices with the vector components of the axes
#As, Bs are arrays of the a and b values for all three cases

fig, ax = plt.subplots()

width_all, height_all = 2*np.linalg.norm(all[0]), 2*np.linalg.norm(all[1])
angle_all = np.degrees(np.arctan2(all[0][0], all[1][0]))
center = (As[0],Bs[0])
r = np.radians(angle_all)

ellipse_case1 = Ellipse(center, width_all, height_all, angle = angle_all, facecolor = 'none', edgecolor = 'green')

ax.add_patch(ellipse_case1)

ax1_points = [
    (center[0] + width_all/2 * cos(r), center[1] + width_all/2 * sin(r)),
    (center[0] - width_all/2 * cos(r), center[1] - width_all/2 * sin(r))
]

ax2_points = [
    (center[0] + height_all/2 * cos(r + radians(90)), center[1] + height_all/2 * sin(r + radians(90))),
    (center[0] + height_all/2 * cos(r + radians(270)), center[1] + height_all/2 * sin(r + radians(270)))
]


arrowprops = dict(arrowstyle="<->", color="blue", linewidth=1.5)
ax.annotate("", xy=ax1_points[0], xytext=ax1_points[1], arrowprops=arrowprops)
ax.annotate("", xy=ax2_points[0], xytext=ax2_points[1], arrowprops=arrowprops)
ax.set_xlim(center[0] - width_all, center[0] + width_all)
ax.set_ylim(center[1] - height_all, center[1] + height_all)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title('"All" case ellipse')
plt.show() 

In [ ]:
print(all)

In [ ]:
print("all case a,b:", As[0], Bs[0])
print("early case a,b:", As[1], Bs[1])
print("late case a,b:", As[2], Bs[2])